# Construct training and testing dataset from simulation history

In [110]:
import json
import os
import datetime
import pandas as pd
import numpy as np

In [123]:
# get files under fold path
def get_file_path_list(fold_path):
    file_list = []
    files = os.listdir(fold_path)
    for f in files:
        f_path = fold_path + f
        file_list.append(f_path)
    return file_list

In [124]:
# read json function
def read_json_from_file(file_name):
    with open(file_name, 'r') as fd:
        data = fd.read()
    return json.loads(data)

## Read order update history

### Functions

In [125]:
def read_json_to_df(file_path):
    data = read_json_from_file(file_path)
    df = pd.DataFrame(columns=['order_id', 'creation_time', 'committed_completion_time', 'pickup_time', 'completion_time'])
    df['order_id'] = data.keys()
    
    creation_time, committed_completion_time, pickup_time, completion_time = [], [], [], []
    for order_id in data.keys():
        creation_time.append(data[order_id][1]['update_time'])
        committed_completion_time.append(data[order_id][0]['committed_completion_time'])
        pickup_time.append(data[order_id][2]['update_time'])
        completion_time.append(data[order_id][3]['update_time'])
    
    # DataFrame
    df['creation_time'] = creation_time
    df['committed_completion_time'] = committed_completion_time
    df['pickup_time'] = pickup_time
    df['completion_time'] = completion_time
    df[['creation_time', 'committed_completion_time', 'pickup_time', 'completion_time']] = \
    df[['creation_time', 'committed_completion_time', 'pickup_time', 'completion_time']].apply(pd.to_datetime) # transfer to datetime
    df['creation_time'] = df['creation_time'].dt.time
    df['committed_completion_time'] = df['committed_completion_time'].dt.time
    df['pickup_time'] = df['pickup_time'].dt.time
    df['completion_time'] = df['completion_time'].dt.time
    
    return df

In [126]:
def period_id_to_dict(file_path):
    order_per_time_period = pd.read_csv(file_path)
    order_per_time_period.columns = ['time_period_id', 'create_time_period', 'number']
    order_per_time_period = order_per_time_period.loc[(order_per_time_period['time_period_id'] >=4) & (order_per_time_period['time_period_id'] <= 11)].reset_index(drop=True)
    order_per_time_period['create_time_period'] = order_per_time_period['create_time_period'].apply(pd.to_datetime)
    order_per_time_period['add_time'] = order_per_time_period['create_time_period'] + pd.Timedelta(minutes=45)
    order_per_time_period['create_time_period'] = order_per_time_period['create_time_period'].dt.time
    order_per_time_period['add_time'] = order_per_time_period['add_time'].dt.time
    # convert to dict
    period_id_to_time_dict = pd.Series(order_per_time_period.create_time_period.values, \
                                       index=order_per_time_period.time_period_id.values).to_dict()
    period_id_to_add_time_dict = pd.Series(order_per_time_period.add_time.values, \
                                           index=order_per_time_period.time_period_id.values).to_dict()
    return period_id_to_time_dict, period_id_to_add_time_dict

In [127]:
def update_dataframe(dataframe, dict_1, dict_2):
    dataframe['time_period_id'] = 0
    for index, row in dataframe.iterrows():
        for period_id in dict_1:
            # combine today to compare
            row_time = datetime.datetime.combine(datetime.date.today(), row['creation_time'])
            start_time = datetime.datetime.combine(datetime.date.today(), dict_1[period_id])
            end_time = datetime.datetime.combine(datetime.date.today(), dict_2[period_id])
            # update time period id
            if row_time >= start_time and row_time <= end_time:
                dataframe.at[index, 'time_period_id'] = period_id
                continue
    # update dataframe
    for index, row in dataframe.iterrows():
        dataframe.at[index, 'creation_time'] = datetime.datetime.combine(datetime.date.today(), row['creation_time'])
        dataframe.at[index, 'committed_completion_time'] = datetime.datetime.combine(datetime.date.today(), row['committed_completion_time'])
        dataframe.at[index, 'pickup_time'] = datetime.datetime.combine(datetime.date.today(), row['pickup_time'])
        dataframe.at[index, 'completion_time'] = datetime.datetime.combine(datetime.date.today(), row['completion_time'])
    
    # compute overdue & delivery time
    dataframe[['creation_time', 'committed_completion_time', 'pickup_time', 'completion_time']] = \
    dataframe[['creation_time', 'committed_completion_time', 'pickup_time', 'completion_time']].apply(pd.to_datetime)
    dataframe['overdue'] = (dataframe['committed_completion_time'].values.astype(np.int64) - dataframe['completion_time'].values.astype(np.int64)) // 10 ** 9
    dataframe['delivery_time'] = (dataframe['completion_time'].values.astype(np.int64) - dataframe['creation_time'].values.astype(np.int64)) // 10 ** 9
    
    return dataframe

In [252]:
def get_json_file_to_instance_dict(instance_dir:str, json_dir:str):
    '''
    Input:
    - instance_dir: instance file path for instances
    - json_dir: json file path for order update history
    '''
    instance_folder_list = []
    json_file_to_instance_file_dict = {}
    
    # json file path list
    json_file_path_list = get_file_path_list(json_dir)
    json_file_path_list.sort(key=lambda x: os.path.getmtime(x))

    # instance fold list
    for file in os.listdir(instance_dir):
        d = os.path.join(instance_dir, file)
        if os.path.isdir(d):
            instance_folder_list.append(d)

    for json_file in json_file_path_list:
        num_orders = int(json_file.split("/")[-1].split("_")[1][:-1])
        num_drivers = int(json_file.split("/")[-1].split("_")[2][:-1])
        for fold in instance_folder_list:
            driver_len, order_len = 0, 0
            for csv_file in os.listdir(fold):
                if csv_file.startswith("driver"):
                    driver_file_name = csv_file
                    driver_df = pd.read_csv(os.path.join(fold, driver_file_name))
                    driver_len = len(driver_df)
                else:
                    instance_file_name = csv_file
                    instance_file_df = pd.read_csv(os.path.join(fold, instance_file_name))
                    order_len = len(instance_file_df)
            if num_orders == order_len and num_drivers == driver_len:
                json_file_to_instance_file_dict[json_file] = os.path.join(fold, instance_file_name)
   
    return json_file_to_instance_file_dict

In [435]:
%%time
json_file_to_instance_file_dict = get_json_file_to_instance_dict("Food_delivery_simulator/Benchmark/Instance_medium/",\
                                                                 "Food_delivery_simulator/Order_update_history/Instance_medium/")
json_file_to_instance_file_dict

CPU times: user 1min 14s, sys: 7.04 s, total: 1min 21s
Wall time: 1min 22s


{'Food_delivery_simulator/Order_update_history/Instance_medium/Instance_1498o_80d_3051l.json': 'Food_delivery_simulator/Benchmark/Instance_medium/Instance_1/instance_1.csv',
 'Food_delivery_simulator/Order_update_history/Instance_medium/Instance_1498o_85d_3051l.json': 'Food_delivery_simulator/Benchmark/Instance_medium/Instance_2/instance_2.csv',
 'Food_delivery_simulator/Order_update_history/Instance_medium/Instance_1498o_90d_3051l.json': 'Food_delivery_simulator/Benchmark/Instance_medium/Instance_3/instance_3.csv',
 'Food_delivery_simulator/Order_update_history/Instance_medium/Instance_1498o_95d_3051l.json': 'Food_delivery_simulator/Benchmark/Instance_medium/Instance_4/instance_4.csv',
 'Food_delivery_simulator/Order_update_history/Instance_medium/Instance_1498o_100d_3051l.json': 'Food_delivery_simulator/Benchmark/Instance_medium/Instance_5/instance_5.csv',
 'Food_delivery_simulator/Order_update_history/Instance_medium/Instance_1498o_105d_3051l.json': 'Food_delivery_simulator/Benchmar

### Run

In [436]:
%%time
file_path_list = get_file_path_list("Food_delivery_simulator/Order_update_history/Instance_medium/") # need update for different instance
period_id_to_time_dict, period_id_to_add_time_dict = period_id_to_dict("../Data/order_per_time_period_2.csv")

total_Dataframe = pd.DataFrame(columns=['time_period_id', 'overdue', 'ID', 'delivery_time'])

for file_path in file_path_list:
    # read order update json file
    dataframe = read_json_to_df(file_path)
    new_dataframe = update_dataframe(dataframe, period_id_to_time_dict, period_id_to_add_time_dict)
    
    # if overdue > 0, set equals to 0
    for index, row in new_dataframe.iterrows():
        if row['overdue'] > 0:
            new_dataframe.at[index, 'overdue'] = 0
        else:
            continue
    
    # read correspond instance file
    correspond_instance_file = json_file_to_instance_file_dict[file_path]
    correspond_instance_df = pd.read_csv(correspond_instance_file, converters={'order_id': lambda x: str(x)})
    
    # merge to a new dataframe
    new_combined_df = pd.merge(new_dataframe, correspond_instance_df[['order_id', 'pickup_id']], how='inner', on='order_id')
    new_combined_df.sort_values(by=['pickup_id', 'time_period_id'], inplace=True)
    new_combined_df['num_orders'] = 1
    
    # delivery time dataframe
    delivery_time_df = new_combined_df[['pickup_id', 'time_period_id', 'delivery_time']].groupby(['pickup_id', 'time_period_id'], as_index=False).mean()
    delivery_time_df["ID"] = file_path.split("/")[-1].split(".")[0] # get instance file name as id
    
    # overdue time dataframe
    overdue_df = new_combined_df[['pickup_id', 'time_period_id', 'overdue']].groupby(['pickup_id', 'time_period_id'], as_index=False).mean()
    overdue_df["ID"] = file_path.split("/")[-1].split(".")[0]
    
    # number orders dataframe
    num_order_df = new_combined_df[['pickup_id', 'time_period_id', 'num_orders']].groupby(['pickup_id', 'time_period_id'], as_index=False).sum()
    num_order_df["ID"] = file_path.split("/")[-1].split(".")[0]
    
    # merge two dataframe
    merge_df = pd.merge(overdue_df, delivery_time_df, how='inner', on=['pickup_id', 'time_period_id', 'ID'])
    merge_df = pd.merge(merge_df, num_order_df, how='inner', on=['pickup_id', 'time_period_id', 'ID'])
    
    # append to total_Dataframe
    total_Dataframe = total_Dataframe.append(merge_df)

total_Dataframe

CPU times: user 2min 40s, sys: 889 ms, total: 2min 41s
Wall time: 2min 45s


,time_period_id,overdue,ID,delivery_time,pickup_id,num_orders
0,4,0.000000,Instance_1898o_100d_3051l,1239.888889,"36 Boat Quay, Singapore 049825, Singapore",9.0
1,5,0.000000,Instance_1898o_100d_3051l,1197.153846,"36 Boat Quay, Singapore 049825, Singapore",13.0
2,6,-41.857143,Instance_1898o_100d_3051l,1690.428571,"36 Boat Quay, Singapore 049825, Singapore",14.0
3,7,-28.500000,Instance_1898o_100d_3051l,1711.500000,"36 Boat Quay, Singapore 049825, Singapore",12.0
4,8,0.000000,Instance_1898o_100d_3051l,1179.142857,"36 Boat Quay, Singapore 049825, Singapore",7.0
...,...,...,...,...,...,...
115,7,0.000000,Instance_1700o_115d_3051l,772.818182,Zion Riverside Food Centre,11.0
116,8,0.000000,Instance_1700o_115d_3051l,803.500000,Zion Riverside Food Centre,6.0
117,9,0.000000,Instance_1700o_115d_3051l,728.200000,Zion Riverside Food Centre,5.0
118,10,0.000000,Instance_1700o_115d_3051l,957.750000,Zion Riverside Food Centre,4.0


In [257]:
total_Dataframe.to_csv("../Data/Training_testing_data/simulation_data_small.csv",index=False)

## Read driver update history

### Functions

In [320]:
def get_od_pair_distance(route_info_path):
    '''
    Input:
    - route_info_path: route_info.csv file path for instance
    '''
    route_info_df = pd.read_csv(route_info_path)
    route_info_df['od_pair'] = list(zip(route_info_df.start_location_id, route_info_df.end_location_id))
    od_pair_distance_dict = pd.Series(route_info_df.distance.values, index=route_info_df.od_pair.values).to_dict()
    return od_pair_distance_dict

In [326]:
def get_restaurants_list(restaurant_info_path):
    '''
    Input:
    - restaurant_info_path: restaurant_info.csv file path for instance
    '''
    restaurant_df = pd.read_csv(restaurant_info_path)
    restaurant_list = list(restaurant_df['restaurant_id'].unique())
    return restaurant_list

In [363]:
def get_driver_visit_restaurant_frequency(driver_update_json_file_path, restaurant_list, od_pair_distance_dict, dict_1, dict_2):
    '''
    Input:
    - driver_update_json_file_path: driver visiting history json file
    - restaurant_list: restaurants in instance
    - od_pair_distance_dict: {od_pair: distance}
    - dict_1, dict_2: time period id dict
    '''
    driver_history_dict = read_json_from_file(driver_update_json_file_path)
    driver_list = list(driver_history_dict.keys())
    
    columns_list = ['ID', 'driver_id', 'time_period_id'] + restaurant_list
    total_driver_dataframe = pd.DataFrame(columns=columns_list)
    for driver in driver_list:
        # construct dataframe for each driver
        driver_df = pd.DataFrame(driver_history_dict[driver])
        # driver_df['ID'] = driver_update_json_file_path
        driver_df['update_time'] = driver_df['update_time'].apply(pd.to_datetime).dt.time
        driver_df['driver_id'] = driver
        driver_df['time_period_id'] = 0
        
        # construct columns for each restaurant
        for r in restaurant_list:
            driver_df[r] = 0.0

        for index, row in driver_df.iterrows():
            driver_location = row['location_id']

            # get the time peroid id
            for period_id in dict_1:
                # combine today to compare
                row_time = datetime.datetime.combine(datetime.date.today(), row['update_time'])
                start_time = datetime.datetime.combine(datetime.date.today(), dict_1[period_id])
                end_time = datetime.datetime.combine(datetime.date.today(), dict_2[period_id])
                # add time period id
                if row_time >= start_time and row_time <= end_time:
                    driver_df.at[index, 'time_period_id'] = period_id
                    continue

            # in restaurant dispatch area or not
            for r in restaurant_list:
                distance = od_pair_distance_dict[(r, driver_location)]
                if distance <= 5.0:
                    driver_df.at[index, r] = 1
                # devide carrying orders
                if row['carrying_orders'] != 0:
                    driver_df.at[index, r] = driver_df.at[index, r] / row['carrying_orders']
        
        # update driver dataframe
        update_driver_df = driver_df[['driver_id', 'time_period_id']+restaurant_list].groupby(['driver_id', 'time_period_id'], as_index=False).sum()
        visit_times_to_time_period_dict = driver_df['time_period_id'].value_counts().to_dict()
        for index, row in update_driver_df.iterrows():
            for r in restaurant_list:
                time_period = row['time_period_id']
                update_driver_df.at[index, r] = row[r] / visit_times_to_time_period_dict[time_period]
        
        # remove the first row (time_period_id = 0)
        update_driver_df = update_driver_df.iloc[1:,:]
        total_driver_dataframe = total_driver_dataframe.append(update_driver_df)
    
    return total_driver_dataframe

### Run

In [437]:
%%time
# read route and restaurant info json file
od_pair_distance_dict = get_od_pair_distance("Food_delivery_simulator/Benchmark/Instance_medium/route_info.csv") # need udpate for different instance
restaurant_list = get_restaurants_list("Food_delivery_simulator/Benchmark/Instance_medium/restaurant_info.csv") # need udpate for different instance

# read driver update history json file
driver_json_file_path_list = get_file_path_list("Food_delivery_simulator/Driver_update_history/Instance_medium/")

# iterate all the json file in 
all_instance_total_driver_df = pd.DataFrame(columns=['time_period_id']+restaurant_list)
for driver_json_file in driver_json_file_path_list:
    # dataframe for a single json file
    total_driver_dataframe = get_driver_visit_restaurant_frequency(driver_json_file, \
                                                                  restaurant_list, od_pair_distance_dict, period_id_to_time_dict, period_id_to_add_time_dict)
    # take mean value on all driver
    total_driver_dataframe = total_driver_dataframe[['time_period_id']+restaurant_list].groupby(['time_period_id'], as_index=False).mean()
    total_driver_dataframe['ID'] = driver_json_file.split("/")[-1].split(".")[0]
    all_instance_total_driver_df = all_instance_total_driver_df.append(total_driver_dataframe)

all_instance_total_driver_df

CPU times: user 10min 51s, sys: 3.68 s, total: 10min 55s
Wall time: 11min 7s


,time_period_id,Maxwell Food Centre,Hong Lim Food Centre,Old Airport Road Food Centre,"37 Cuppage Rd, Singapore 229460, Singapore",Alexandra Village Food Centre,Tiong Bahru Market,Bukit Merah View Market & Hawker Centre,Amoy Street Food Centre,Golden Mile Food Centre,"66 Kg Bugis, Singapore 338987, Singapore",Zion Riverside Food Centre,"36 Boat Quay, Singapore 049825, Singapore","417 River Valley Rd, Singapore 248316, Singapore","371 Beach Rd, Singapore 199597, Singapore","54 South Bridge Rd, Singapore 058685, Singapore",ID
0,4,0.853913,0.857563,0.381994,0.885310,0.660945,0.864635,0.776730,0.847405,0.829876,0.820112,0.869714,0.869385,0.873048,0.869400,0.909464,Instance_1898o_100d_3051l
1,5,0.799859,0.818668,0.345487,0.826597,0.630756,0.816406,0.749569,0.792656,0.777759,0.763807,0.823323,0.830874,0.821656,0.812492,0.862037,Instance_1898o_100d_3051l
2,6,0.766800,0.780185,0.310807,0.814050,0.643433,0.780355,0.743347,0.762942,0.724040,0.695984,0.796046,0.773091,0.802252,0.753405,0.829784,Instance_1898o_100d_3051l
3,7,0.745988,0.766397,0.324976,0.786446,0.605022,0.762440,0.701756,0.743310,0.726171,0.702512,0.765060,0.776343,0.768710,0.759323,0.814196,Instance_1898o_100d_3051l
4,8,0.737924,0.763659,0.317005,0.797190,0.626607,0.764897,0.721379,0.735353,0.704179,0.686263,0.792809,0.771679,0.802393,0.741883,0.801631,Instance_1898o_100d_3051l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,7,0.787505,0.814662,0.332210,0.840949,0.657420,0.793423,0.758199,0.784607,0.768635,0.728402,0.818420,0.832178,0.818834,0.807210,0.862626,Instance_1700o_115d_3051l
4,8,0.738882,0.753996,0.416271,0.797930,0.563474,0.723685,0.704679,0.736398,0.775974,0.758943,0.755197,0.766839,0.756439,0.806223,0.780790,Instance_1700o_115d_3051l
5,9,0.742019,0.760300,0.421429,0.801087,0.545055,0.723064,0.698178,0.739534,0.751745,0.743111,0.745631,0.763108,0.743820,0.781534,0.786139,Instance_1700o_115d_3051l
6,10,0.751032,0.763178,0.432802,0.797257,0.534938,0.717764,0.689831,0.745131,0.779823,0.757010,0.750518,0.765928,0.746532,0.804561,0.779990,Instance_1700o_115d_3051l


## Combine the order and driver results

In [428]:
all_instance_total_driver_df

,time_period_id,Old Airport Road Food Centre,Kovan Food Centre (209 Hougang Street 21),Bedok Interchange Hawker Centre,Lorong 8 Toa Payoh Hawker Centre,Chomp Chomp,"Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore",ID
0,4,0.501265,0.637608,0.304467,0.781916,0.633923,0.926331,Instance_848o_70d_2165l
1,5,0.418085,0.627535,0.246633,0.634705,0.647993,0.777890,Instance_848o_70d_2165l
2,6,0.439206,0.610187,0.283934,0.606689,0.623175,0.760006,Instance_848o_70d_2165l
3,7,0.450660,0.650681,0.290533,0.561338,0.583271,0.724568,Instance_848o_70d_2165l
4,8,0.431826,0.610475,0.322733,0.507862,0.570262,0.680870,Instance_848o_70d_2165l
...,...,...,...,...,...,...,...,...
3,7,0.416493,0.586825,0.318231,0.562101,0.588783,0.732657,Instance_1050o_105d_2165l
4,8,0.421553,0.550378,0.316629,0.534630,0.531984,0.674240,Instance_1050o_105d_2165l
5,9,0.427230,0.549788,0.331323,0.508080,0.506701,0.658549,Instance_1050o_105d_2165l
6,10,0.395072,0.541935,0.345960,0.462831,0.483500,0.608908,Instance_1050o_105d_2165l


In [429]:
total_Dataframe

,time_period_id,overdue,ID,delivery_time,pickup_id,num_orders
0,4,0,Instance_848o_70d_2165l,915.272727,Bedok Interchange Hawker Centre,22.0
1,5,0,Instance_848o_70d_2165l,1125.333333,Bedok Interchange Hawker Centre,30.0
2,6,0,Instance_848o_70d_2165l,1136.484848,Bedok Interchange Hawker Centre,33.0
3,7,0,Instance_848o_70d_2165l,959.551724,Bedok Interchange Hawker Centre,29.0
4,8,0,Instance_848o_70d_2165l,1093.764706,Bedok Interchange Hawker Centre,17.0
...,...,...,...,...,...,...
43,7,0,Instance_1050o_105d_2165l,1008.767442,Old Airport Road Food Centre,43.0
44,8,0,Instance_1050o_105d_2165l,1028.000000,Old Airport Road Food Centre,25.0
45,9,0,Instance_1050o_105d_2165l,973.550000,Old Airport Road Food Centre,20.0
46,10,0,Instance_1050o_105d_2165l,1070.937500,Old Airport Road Food Centre,16.0


In [431]:
%%time
res_to_nearby_res_num_dict = {'Kovan Food Centre (209 Hougang Street 21)': 3,
                   'Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore': 5,
                   'Bedok Interchange Hawker Centre': 1,
                   'Chomp Chomp': 3,
                   'Lorong 8 Toa Payoh Hawker Centre': 4,
                   'Old Airport Road Food Centre': 2}
res_to_id_dict = {'Kovan Food Centre (209 Hougang Street 21)': 1,
                   'Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore': 2,
                   'Bedok Interchange Hawker Centre': 3,
                   'Chomp Chomp': 4,
                   'Lorong 8 Toa Payoh Hawker Centre': 5,
                   'Old Airport Road Food Centre': 6}
res_order_driver_history_df_list = []

for r in restaurant_list:
    driver_history__df = all_instance_total_driver_df[['ID', 'time_period_id']+[r]]
    order_history_df = total_Dataframe[total_Dataframe['pickup_id'] == r]
    merge_order_driver_history_df = pd.merge(driver_history__df, order_history_df[['ID', 'time_period_id', 'num_orders', 'overdue', 'delivery_time']], \
                                             how='inner', on=['ID', 'time_period_id'])
    # change the column name
    merge_order_driver_history_df = merge_order_driver_history_df.rename(columns={r: 'number_of_driver_index'})
    merge_order_driver_history_df['restaurant_id'] =  res_to_id_dict[r]
    merge_order_driver_history_df['nearby_restaurant'] = res_to_nearby_res_num_dict[r] # add nearby restaurant column
    res_order_driver_history_df_list.append(merge_order_driver_history_df)
    # print(merge_order_driver_history_df)
    # break

 # write to csv file
output_df = pd.concat(res_order_driver_history_df_list, ignore_index=True) 
output_df.to_csv(f"../Data/Training_testing_data/Instance_small_2/output_data.csv") # need to update for different instance

CPU times: user 85.1 ms, sys: 14.3 ms, total: 99.4 ms
Wall time: 124 ms


In [433]:
output_df

,ID,time_period_id,number_of_driver_index,num_orders,overdue,delivery_time,restaurant_id,nearby_restaurant
0,Instance_848o_70d_2165l,4,0.501265,27.0,0,1045.111111,6,2
1,Instance_848o_70d_2165l,5,0.418085,37.0,0,1062.378378,6,2
2,Instance_848o_70d_2165l,6,0.439206,41.0,0,1052.390244,6,2
3,Instance_848o_70d_2165l,7,0.450660,35.0,0,997.228571,6,2
4,Instance_848o_70d_2165l,8,0.431826,20.0,0,1017.500000,6,2
...,...,...,...,...,...,...,...,...
3835,Instance_1050o_105d_2165l,7,0.732657,16.0,0,1066.000000,2,5
3836,Instance_1050o_105d_2165l,8,0.674240,10.0,0,1130.400000,2,5
3837,Instance_1050o_105d_2165l,9,0.658549,8.0,0,1250.125000,2,5
3838,Instance_1050o_105d_2165l,10,0.608908,6.0,0,1359.666667,2,5


In [426]:
80 * 8 * 6

3840

## Test Instance on driver history data

In [258]:
%%time
route_info_df = pd.read_csv("Food_delivery_simulator/Benchmark/Instance_small/route_info.csv")
route_info_df.head()

CPU times: user 806 ms, sys: 151 ms, total: 957 ms
Wall time: 1.47 s


,route_code,start_location_id,end_location_id,distance,time
0,0,Alexandra Village Food Centre,Alexandra Village Food Centre,0.00,0
1,1,Alexandra Village Food Centre,Ghim Moh Market & Food Centre,3.28,393
2,2,Alexandra Village Food Centre,Bukit Timah Market & Food Centre,6.72,806
3,3,Alexandra Village Food Centre,276425 Singapore,4.61,553
4,4,Alexandra Village Food Centre,118676 Singapore,3.10,372


In [296]:
route_info_df['od_pair'] = list(zip(route_info_df.start_location_id, route_info_df.end_location_id))
route_info_df.head()

,route_code,start_location_id,end_location_id,distance,time,od_pair
0,0,Alexandra Village Food Centre,Alexandra Village Food Centre,0.00,0,"(Alexandra Village Food Centre, Alexandra Vill..."
1,1,Alexandra Village Food Centre,Ghim Moh Market & Food Centre,3.28,393,"(Alexandra Village Food Centre, Ghim Moh Marke..."
2,2,Alexandra Village Food Centre,Bukit Timah Market & Food Centre,6.72,806,"(Alexandra Village Food Centre, Bukit Timah Ma..."
3,3,Alexandra Village Food Centre,276425 Singapore,4.61,553,"(Alexandra Village Food Centre, 276425 Singapore)"
4,4,Alexandra Village Food Centre,118676 Singapore,3.10,372,"(Alexandra Village Food Centre, 118676 Singapore)"


In [297]:
od_pair_distance_dict = pd.Series(route_info_df.distance.values, index=route_info_df.od_pair.values).to_dict()
od_pair_distance_dict

{('Alexandra Village Food Centre', 'Alexandra Village Food Centre'): 0.0,
 ('Alexandra Village Food Centre', 'Ghim Moh Market & Food Centre'): 3.28,
 ('Alexandra Village Food Centre', 'Bukit Timah Market & Food Centre'): 6.72,
 ('Alexandra Village Food Centre', '276425 Singapore'): 4.61,
 ('Alexandra Village Food Centre', '118676 Singapore'): 3.1,
 ('Alexandra Village Food Centre', '589485 Singapore'): 6.39,
 ('Alexandra Village Food Centre', '141090 Singapore'): 1.7,
 ('Alexandra Village Food Centre', '120413 Singapore'): 4.64,
 ('Alexandra Village Food Centre', '159028 Singapore'): 2.1,
 ('Alexandra Village Food Centre', '159026 Singapore'): 1.97,
 ('Alexandra Village Food Centre', '159553 Singapore'): 1.69,
 ('Alexandra Village Food Centre', '141095 Singapore'): 1.38,
 ('Alexandra Village Food Centre', '276692 Singapore'): 4.53,
 ('Alexandra Village Food Centre', '276697 Singapore'): 4.44,
 ('Alexandra Village Food Centre', '128044 Singapore'): 4.46,
 ('Alexandra Village Food Centre

In [172]:
driver_history_dict = read_json_from_file("Food_delivery_simulator/Driver_update_history/Instance_699o_25d_1158l.json")
driver_history_dict['D_1']

[{'carrying_orders': 0,
  'location_id': 'Bukit Timah Market & Food Centre',
  'update_time': '02/16/2022, 09:00:00'},
 {'carrying_orders': 0,
  'location_id': 'Bukit Timah Market & Food Centre',
  'update_time': '02/16/2022, 09:00:00'},
 {'carrying_orders': 0,
  'location_id': 'Bukit Timah Market & Food Centre',
  'update_time': '02/16/2022, 09:10:00'},
 {'carrying_orders': 0,
  'location_id': 'Bukit Timah Market & Food Centre',
  'update_time': '02/16/2022, 09:11:00'},
 {'carrying_orders': 1,
  'location_id': '650406 Singapore',
  'update_time': '02/16/2022, 09:20:49'},
 {'carrying_orders': 0,
  'location_id': 'Bukit Timah Market & Food Centre',
  'update_time': '02/16/2022, 09:30:38'},
 {'carrying_orders': 0,
  'location_id': '598737 Singapore',
  'update_time': '02/16/2022, 09:35:58'},
 {'carrying_orders': 2,
  'location_id': '598737 Singapore',
  'update_time': '02/16/2022, 09:40:00'},
 {'carrying_orders': 2,
  'location_id': 'Bukit Timah Market & Food Centre',
  'update_time': '0

In [325]:
list(driver_history_dict.keys())

['D_1',
 'D_2',
 'D_3',
 'D_4',
 'D_5',
 'D_6',
 'D_7',
 'D_8',
 'D_9',
 'D_10',
 'D_11',
 'D_12',
 'D_13',
 'D_14',
 'D_15',
 'D_16',
 'D_17',
 'D_18',
 'D_19',
 'D_20',
 'D_21',
 'D_22',
 'D_23',
 'D_24',
 'D_25']

In [316]:
driver_df = pd.DataFrame(driver_history_dict['D_1'])
driver_df['update_time'] = driver_df['update_time'].apply(pd.to_datetime).dt.time
driver_df.head()

,carrying_orders,location_id,update_time
0,0,Bukit Timah Market & Food Centre,09:00:00
1,0,Bukit Timah Market & Food Centre,09:00:00
2,0,Bukit Timah Market & Food Centre,09:10:00
3,0,Bukit Timah Market & Food Centre,09:11:00
4,1,650406 Singapore,09:20:49


In [279]:
restaurant_df = pd.read_csv("Food_delivery_simulator/Benchmark/Instance_small/restaurant_info.csv")
restaurant_list = list(restaurant_df['restaurant_id'].unique())
restaurant_list

['Ghim Moh Market & Food Centre',
 'Bukit Timah Market & Food Centre',
 'Alexandra Village Food Centre']

In [354]:
%%time
driver_df['time_period_id'] = 0

for r in restaurant_list:
    driver_df[r] = 0.0

for index, row in driver_df.iterrows():
    driver_location = row['location_id']

    # get the time peroid id
    for period_id in period_id_to_time_dict:
        # combine today to compare
        row_time = datetime.datetime.combine(datetime.date.today(), row['update_time'])
        start_time = datetime.datetime.combine(datetime.date.today(), period_id_to_time_dict[period_id])
        end_time = datetime.datetime.combine(datetime.date.today(), period_id_to_add_time_dict[period_id])
        # add time period id
        if row_time >= start_time and row_time <= end_time:
            driver_df.at[index, 'time_period_id'] = period_id
            continue
    
    # in restaurant dispatch area or not
    for r in restaurant_list:
        distance = od_pair_distance_dict[(r, driver_location)]
        if distance <= 5.0:
            driver_df.at[index, r] = 1.0 
        # devide carrying orders
        if row['carrying_orders'] != 0:
            driver_df.at[index, r] = driver_df.at[index, r] / row['carrying_orders']

driver_df

CPU times: user 21.2 ms, sys: 1.84 ms, total: 23 ms
Wall time: 23.7 ms


,carrying_orders,location_id,update_time,time_period_id,Ghim Moh Market & Food Centre,Bukit Timah Market & Food Centre,Alexandra Village Food Centre
0,0,Bukit Timah Market & Food Centre,09:00:00,4,1.000000,1.000000,0.000000
1,0,Bukit Timah Market & Food Centre,09:00:00,4,1.000000,1.000000,0.000000
2,0,Bukit Timah Market & Food Centre,09:10:00,4,1.000000,1.000000,0.000000
3,0,Bukit Timah Market & Food Centre,09:11:00,4,1.000000,1.000000,0.000000
4,1,650406 Singapore,09:20:49,4,0.000000,1.000000,0.000000
5,0,Bukit Timah Market & Food Centre,09:30:38,4,1.000000,1.000000,0.000000
6,0,598737 Singapore,09:35:58,4,0.000000,1.000000,0.000000
7,2,598737 Singapore,09:40:00,4,0.000000,0.500000,0.000000
8,2,Bukit Timah Market & Food Centre,09:46:20,5,0.500000,0.500000,0.000000
9,1,651442 Singapore,09:56:28,5,0.000000,1.000000,0.000000


In [355]:
update_driver_df = driver_df[['time_period_id', 'Ghim Moh Market & Food Centre', 'Bukit Timah Market & Food Centre', \
           'Alexandra Village Food Centre']].groupby(['time_period_id'], as_index=False).sum()
visit_times_to_time_period_dict = driver_df['time_period_id'].value_counts().to_dict()

for index, row in update_driver_df.iterrows():
    for r in restaurant_list:
        time_period = row['time_period_id']
        update_driver_df.at[index, r] = row[r] / visit_times_to_time_period_dict[time_period]

update_driver_df = update_driver_df.iloc[1:,:]
update_driver_df

,time_period_id,Ghim Moh Market & Food Centre,Bukit Timah Market & Food Centre,Alexandra Village Food Centre
1,4,0.625000,0.937500,0.000000
2,5,0.166667,0.666667,0.000000
3,6,0.398148,0.398148,0.185185
4,7,0.523810,0.595238,0.285714
5,8,0.472222,0.263889,0.305556
6,9,1.000000,1.000000,0.250000
7,10,0.900000,0.600000,0.900000
8,11,1.000000,0.714286,0.714286


In [432]:
visit_times_to_time_period_dict

{6: 9, 4: 8, 9: 8, 7: 7, 11: 7, 8: 6, 5: 5, 10: 5, 0: 2}

## Test Instance on order history data

In [405]:
test_df = read_json_to_df("Food_delivery_simulator/Order_update_history/Instance_699o_25d_1158l.json")
test_df = update_dataframe(test_df, period_id_to_time_dict, period_id_to_add_time_dict)
test_df

,order_id,creation_time,committed_completion_time,pickup_time,completion_time,time_period_id,overdue,delivery_time
0,09002800000,2022-02-20 09:00:28,2022-02-20 09:45:28,2022-02-20 09:10:00,2022-02-20 09:17:07,4,1701,999
1,09004600001,2022-02-20 09:00:46,2022-02-20 09:45:46,2022-02-20 09:16:56,2022-02-20 09:23:39,4,1327,1373
2,09013000002,2022-02-20 09:01:30,2022-02-20 09:46:30,2022-02-20 09:10:00,2022-02-20 09:19:37,4,1613,1087
3,09013900003,2022-02-20 09:01:39,2022-02-20 09:46:39,2022-02-20 09:10:00,2022-02-20 09:12:39,4,2040,660
4,09014100004,2022-02-20 09:01:41,2022-02-20 09:46:41,2022-02-20 09:16:33,2022-02-20 09:23:25,4,1396,1304
...,...,...,...,...,...,...,...,...
694,14545500694,2022-02-20 14:54:55,2022-02-20 15:39:55,2022-02-20 15:06:15,2022-02-20 15:09:12,11,1843,857
695,14550900695,2022-02-20 14:55:09,2022-02-20 15:40:09,2022-02-20 15:07:49,2022-02-20 15:18:32,11,1297,1403
696,14553600696,2022-02-20 14:55:36,2022-02-20 15:40:36,2022-02-20 15:06:48,2022-02-20 15:13:26,11,1630,1070
697,14560900697,2022-02-20 14:56:09,2022-02-20 15:41:09,2022-02-20 15:04:04,2022-02-20 15:12:28,11,1721,979


In [148]:
test_df['overdue']

0      1726
1      1718
2      1836
3      1986
4      1602
       ... 
492    1371
493    1601
494    1821
495    1284
496    1680
Name: overdue, Length: 497, dtype: int64

In [149]:
for index, row in test_df.iterrows():
    if row['overdue'] > 0:
        test_df.at[index, 'overdue'] = 0
    else:
        continue

test_df

,order_id,creation_time,committed_completion_time,pickup_time,completion_time,time_period_id,overdue,delivery_time
0,09020400000,2022-02-17 09:02:04,2022-02-17 09:47:04,2022-02-17 09:10:00,2022-02-17 09:18:18,4,0,974
1,09040000001,2022-02-17 09:04:00,2022-02-17 09:49:00,2022-02-17 09:16:56,2022-02-17 09:20:22,4,0,982
2,09042200002,2022-02-17 09:04:22,2022-02-17 09:49:22,2022-02-17 09:10:00,2022-02-17 09:18:46,4,0,864
3,09045800003,2022-02-17 09:04:58,2022-02-17 09:49:58,2022-02-17 09:10:00,2022-02-17 09:16:52,4,0,714
4,09060600004,2022-02-17 09:06:06,2022-02-17 09:51:06,2022-02-17 09:16:33,2022-02-17 09:24:24,4,0,1098
...,...,...,...,...,...,...,...,...
492,14532900492,2022-02-17 14:53:29,2022-02-17 15:38:29,2022-02-17 15:09:26,2022-02-17 15:15:38,11,0,1329
493,14541200493,2022-02-17 14:54:12,2022-02-17 15:39:12,2022-02-17 15:08:01,2022-02-17 15:12:31,11,0,1099
494,14551900494,2022-02-17 14:55:19,2022-02-17 15:40:19,2022-02-17 15:04:38,2022-02-17 15:09:58,11,0,879
495,14561800495,2022-02-17 14:56:18,2022-02-17 15:41:18,2022-02-17 15:09:26,2022-02-17 15:19:54,11,0,1416


In [159]:
overdue_group_df = test_df[['time_period_id', 'overdue']].groupby(['time_period_id'], as_index=False).mean()
overdue_group_df

,time_period_id,overdue
0,4,1575.893617
1,5,610.929134
2,6,-45.435714
3,7,-60.883333
4,8,435.485714
5,9,975.339286
6,10,1554.545455
7,11,1701.333333


In [154]:
overdue_df = test_df.loc[test_df['overdue'] < 0]
overdue_df

,order_id,creation_time,committed_completion_time,pickup_time,completion_time,time_period_id,overdue,delivery_time
59,09280400059,2022-02-17 09:28:04,2022-02-17 10:13:04,2022-02-17 09:40:28,2022-02-17 10:49:04,4,-2160,4860
102,09465900102,2022-02-17 09:46:59,2022-02-17 10:31:59,2022-02-17 09:59:22,2022-02-17 10:41:25,5,-566,3266
104,09482700104,2022-02-17 09:48:27,2022-02-17 10:33:27,2022-02-17 10:15:25,2022-02-17 10:58:49,5,-1522,4222
121,09570000121,2022-02-17 09:57:00,2022-02-17 10:42:00,2022-02-17 10:19:27,2022-02-17 11:06:53,5,-1493,4193
129,10005300129,2022-02-17 10:00:53,2022-02-17 10:45:53,2022-02-17 10:17:10,2022-02-17 11:03:48,5,-1075,3775
...,...,...,...,...,...,...,...,...
548,12430400548,2022-02-17 12:43:04,2022-02-17 13:28:04,2022-02-17 13:23:22,2022-02-17 13:30:40,8,-156,2856
549,12431900549,2022-02-17 12:43:19,2022-02-17 13:28:19,2022-02-17 12:55:09,2022-02-17 13:32:59,8,-280,2980
553,12462300553,2022-02-17 12:46:23,2022-02-17 13:31:23,2022-02-17 13:18:54,2022-02-17 13:32:52,9,-89,2789
556,12494800556,2022-02-17 12:49:48,2022-02-17 13:34:48,2022-02-17 13:26:44,2022-02-17 14:02:31,9,-1663,4363


In [160]:
delivery_group_df = test_df[['time_period_id', 'delivery_time']].groupby(['time_period_id'], as_index=False).mean()
delivery_group_df

,time_period_id,delivery_time
0,4,1124.106383
1,5,2089.070866
2,6,2745.435714
3,7,2760.883333
4,8,2264.514286
5,9,1724.660714
6,10,1145.454545
7,11,998.666667


In [161]:
merge_df = pd.merge(overdue_group_df, delivery_group_df, how='inner', on='time_period_id')
merge_df

,time_period_id,overdue,delivery_time
0,4,1575.893617,1124.106383
1,5,610.929134,2089.070866
2,6,-45.435714,2745.435714
3,7,-60.883333,2760.883333
4,8,435.485714,2264.514286
5,9,975.339286,1724.660714
6,10,1554.545455,1145.454545
7,11,1701.333333,998.666667


In [179]:
file_path_list = get_file_path_list("Food_delivery_simulator/Order_update_history/")
file_path_list

['Food_delivery_simulator/Order_update_history/Instance_497o_35d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_497o_45d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_699o_30d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_699o_40d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_598o_30d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_598o_40d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_851o_85d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_598o_25d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_598o_55d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_949o_50d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_699o_25d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_900o_80d_1158l.json',
 'Food_delivery_simulator/Order_update_history/Instance_699o_55d

In [188]:
file_path_list.sort(key=lambda x: os.path.getmtime(x))
for file in file_path_list:
    print(file.split("/")[-1].split("_")[1][:-1], end='')
    print(" ", end='')
    print(file.split("/")[-1].split("_")[2][:-1])

497 25
497 30
497 35
497 40
497 45
497 50
497 55
497 60
548 25
548 30
548 35
548 40
548 45
548 50
548 55
548 60
598 25
598 30
598 35
598 40
598 45
598 50
598 55
598 60
650 25
650 30
650 35
650 40
650 45
650 50
650 55
650 60
699 25
699 30
699 35
699 40
699 45
699 50
699 55
699 60
749 50
749 55
749 60
749 65
749 70
749 75
749 80
749 85
798 50
798 55
798 60
798 65
798 70
798 75
798 80
798 85
851 50
851 55
851 60
851 65
851 70
851 75
851 80
851 85
900 50
900 55
900 60
900 65
900 70
900 75
900 80
900 85
949 50
949 55
949 60
949 65
949 70
949 75
949 80
949 85


In [221]:
%%time
rootdir = "Food_delivery_simulator/Benchmark/Instance_small"
instance_folder_list = []
json_file_to_instance_file_dict = {}

# instance fold list
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        instance_folder_list.append(d)

for json_file in file_path_list:
    num_orders = int(json_file.split("/")[-1].split("_")[1][:-1])
    num_drivers = int(json_file.split("/")[-1].split("_")[2][:-1])
    for fold in instance_folder_list:
        driver_len, order_len = 0, 0
        for csv_file in os.listdir(fold):
            if csv_file.startswith("driver"):
                driver_file_name = csv_file
                driver_df = pd.read_csv(os.path.join(fold, driver_file_name))
                driver_len = len(driver_df)
            else:
                instance_file_name = csv_file
                instance_file_df = pd.read_csv(os.path.join(fold, instance_file_name))
                order_len = len(instance_file_df)
        if num_orders == order_len and num_drivers == driver_len:
            json_file_to_instance_file_dict[json_file] = os.path.join(fold, instance_file_name)

CPU times: user 45.4 s, sys: 3.42 s, total: 48.8 s
Wall time: 50.5 s


In [222]:
json_file_to_instance_file_dict

{'Food_delivery_simulator/Order_update_history/Instance_497o_25d_1158l.json': 'Food_delivery_simulator/Benchmark/Instance_small/Instance_1/instance_1.csv',
 'Food_delivery_simulator/Order_update_history/Instance_497o_30d_1158l.json': 'Food_delivery_simulator/Benchmark/Instance_small/Instance_2/instance_2.csv',
 'Food_delivery_simulator/Order_update_history/Instance_497o_35d_1158l.json': 'Food_delivery_simulator/Benchmark/Instance_small/Instance_3/instance_3.csv',
 'Food_delivery_simulator/Order_update_history/Instance_497o_40d_1158l.json': 'Food_delivery_simulator/Benchmark/Instance_small/Instance_4/instance_4.csv',
 'Food_delivery_simulator/Order_update_history/Instance_497o_45d_1158l.json': 'Food_delivery_simulator/Benchmark/Instance_small/Instance_5/instance_5.csv',
 'Food_delivery_simulator/Order_update_history/Instance_497o_50d_1158l.json': 'Food_delivery_simulator/Benchmark/Instance_small/Instance_6/instance_6.csv',
 'Food_delivery_simulator/Order_update_history/Instance_497o_55d

In [217]:
test_df

,order_id,creation_time,committed_completion_time,pickup_time,completion_time,time_period_id,overdue,delivery_time
0,09002800000,2022-02-17 09:00:28,2022-02-17 09:45:28,2022-02-17 09:10:00,2022-02-17 09:17:07,4,1701,999
1,09004600001,2022-02-17 09:00:46,2022-02-17 09:45:46,2022-02-17 09:16:56,2022-02-17 09:23:39,4,1327,1373
2,09013000002,2022-02-17 09:01:30,2022-02-17 09:46:30,2022-02-17 09:10:00,2022-02-17 09:19:37,4,1613,1087
3,09013900003,2022-02-17 09:01:39,2022-02-17 09:46:39,2022-02-17 09:10:00,2022-02-17 09:12:39,4,2040,660
4,09014100004,2022-02-17 09:01:41,2022-02-17 09:46:41,2022-02-17 09:16:33,2022-02-17 09:23:25,4,1396,1304
...,...,...,...,...,...,...,...,...
694,14545500694,2022-02-17 14:54:55,2022-02-17 15:39:55,2022-02-17 15:06:15,2022-02-17 15:09:12,11,1843,857
695,14550900695,2022-02-17 14:55:09,2022-02-17 15:40:09,2022-02-17 15:07:49,2022-02-17 15:18:32,11,1297,1403
696,14553600696,2022-02-17 14:55:36,2022-02-17 15:40:36,2022-02-17 15:06:48,2022-02-17 15:13:26,11,1630,1070
697,14560900697,2022-02-17 14:56:09,2022-02-17 15:41:09,2022-02-17 15:04:04,2022-02-17 15:12:28,11,1721,979


In [232]:
correspond_instance_file = json_file_to_instance_file_dict["Food_delivery_simulator/Order_update_history/Instance_699o_25d_1158l.json"]
correspond_instance_df = pd.read_csv(correspond_instance_file, converters={'order_id': lambda x: str(x)})
correspond_instance_df

,order_id,demand,creation_time,committed_completion_time,load_time,unload_time,pickup_id,delivery_id,restaurant_name,customer_name,pick_up_coordinate,delivery_coordinate
0,09002800000,1,09:00:28,09:45:28,60,60,Ghim Moh Market & Food Centre,117505 Singapore,Ghim Moh Market & Food Centre,117505 Singapore,"[1.31091018371375, 103.788241408454]","[1.2918101, 103.7684714]"
1,09004600001,1,09:00:46,09:45:46,60,60,Ghim Moh Market & Food Centre,128035 Singapore,Ghim Moh Market & Food Centre,128035 Singapore,"[1.31091018371375, 103.788241408454]","[1.298645899, 103.765644]"
2,09013000002,1,09:01:30,09:46:30,60,60,Bukit Timah Market & Food Centre,289604 Singapore,Bukit Timah Market & Food Centre,289604 Singapore,"[1.33960974933939, 103.776088853304]","[1.324324527, 103.8117093]"
3,09013900003,1,09:01:39,09:46:39,60,60,Ghim Moh Market & Food Centre,271002 Singapore,Ghim Moh Market & Food Centre,271002 Singapore,"[1.31091018371375, 103.788241408454]","[1.309344983, 103.7955594]"
4,09014100004,1,09:01:41,09:46:41,60,60,Ghim Moh Market & Food Centre,126779 Singapore,Ghim Moh Market & Food Centre,126779 Singapore,"[1.31091018371375, 103.788241408454]","[1.299170078, 103.7645453]"
...,...,...,...,...,...,...,...,...,...,...,...,...
694,14545500694,1,14:54:55,15:39:55,60,60,Ghim Moh Market & Food Centre,130027 Singapore,Ghim Moh Market & Food Centre,130027 Singapore,"[1.31091018371375, 103.788241408454]","[1.305422783, 103.7813204]"
695,14550900695,1,14:55:09,15:40:09,60,60,Bukit Timah Market & Food Centre,258853 Singapore,Bukit Timah Market & Food Centre,258853 Singapore,"[1.33960974933939, 103.776088853304]","[1.309843135, 103.808137]"
696,14553600696,1,14:55:36,15:40:36,60,60,Ghim Moh Market & Food Centre,122312 Singapore,Ghim Moh Market & Food Centre,122312 Singapore,"[1.31091018371375, 103.788241408454]","[1.31967297, 103.7644555]"
697,14560900697,1,14:56:09,15:41:09,60,60,Ghim Moh Market & Food Centre,587916 Singapore,Ghim Moh Market & Food Centre,587916 Singapore,"[1.31091018371375, 103.788241408454]","[1.344213673, 103.7882792]"


In [242]:
new_test_df = pd.merge(test_df, correspond_instance_df[['order_id', 'pickup_id']], how='inner', on='order_id')
new_test_df

,order_id,creation_time,committed_completion_time,pickup_time,completion_time,time_period_id,overdue,delivery_time,pickup_id
0,09002800000,2022-02-17 09:00:28,2022-02-17 09:45:28,2022-02-17 09:10:00,2022-02-17 09:17:07,4,1701,999,Ghim Moh Market & Food Centre
1,09004600001,2022-02-17 09:00:46,2022-02-17 09:45:46,2022-02-17 09:16:56,2022-02-17 09:23:39,4,1327,1373,Ghim Moh Market & Food Centre
2,09013000002,2022-02-17 09:01:30,2022-02-17 09:46:30,2022-02-17 09:10:00,2022-02-17 09:19:37,4,1613,1087,Bukit Timah Market & Food Centre
3,09013900003,2022-02-17 09:01:39,2022-02-17 09:46:39,2022-02-17 09:10:00,2022-02-17 09:12:39,4,2040,660,Ghim Moh Market & Food Centre
4,09014100004,2022-02-17 09:01:41,2022-02-17 09:46:41,2022-02-17 09:16:33,2022-02-17 09:23:25,4,1396,1304,Ghim Moh Market & Food Centre
...,...,...,...,...,...,...,...,...,...
694,14545500694,2022-02-17 14:54:55,2022-02-17 15:39:55,2022-02-17 15:06:15,2022-02-17 15:09:12,11,1843,857,Ghim Moh Market & Food Centre
695,14550900695,2022-02-17 14:55:09,2022-02-17 15:40:09,2022-02-17 15:07:49,2022-02-17 15:18:32,11,1297,1403,Bukit Timah Market & Food Centre
696,14553600696,2022-02-17 14:55:36,2022-02-17 15:40:36,2022-02-17 15:06:48,2022-02-17 15:13:26,11,1630,1070,Ghim Moh Market & Food Centre
697,14560900697,2022-02-17 14:56:09,2022-02-17 15:41:09,2022-02-17 15:04:04,2022-02-17 15:12:28,11,1721,979,Ghim Moh Market & Food Centre


In [244]:
cols = new_test_df.columns.to_list()
cols = [cols[0]] + [cols[-1]] + [cols[5]] + cols[1:5] + cols[6:8]
cols

['order_id',
 'pickup_id',
 'time_period_id',
 'creation_time',
 'committed_completion_time',
 'pickup_time',
 'completion_time',
 'overdue',
 'delivery_time']

In [247]:
new_test_df = new_test_df[cols]
new_test_df.sort_values(by=['pickup_id', 'time_period_id'], inplace=True)
new_test_df

,order_id,pickup_id,time_period_id,creation_time,committed_completion_time,pickup_time,completion_time,overdue,delivery_time
6,09022400006,Alexandra Village Food Centre,4,2022-02-17 09:02:24,2022-02-17 09:47:24,2022-02-17 09:10:00,2022-02-17 09:15:51,1893,807
8,09031400008,Alexandra Village Food Centre,4,2022-02-17 09:03:14,2022-02-17 09:48:14,2022-02-17 09:10:00,2022-02-17 09:12:00,2174,526
26,09102200026,Alexandra Village Food Centre,4,2022-02-17 09:10:22,2022-02-17 09:55:22,2022-02-17 09:20:00,2022-02-17 09:27:01,1701,999
31,09140200031,Alexandra Village Food Centre,4,2022-02-17 09:14:02,2022-02-17 09:59:02,2022-02-17 09:25:30,2022-02-17 09:34:33,1469,1231
32,09154800032,Alexandra Village Food Centre,4,2022-02-17 09:15:48,2022-02-17 10:00:48,2022-02-17 09:24:51,2022-02-17 09:34:47,1561,1139
...,...,...,...,...,...,...,...,...,...
693,14532800693,Ghim Moh Market & Food Centre,11,2022-02-17 14:53:28,2022-02-17 15:38:28,2022-02-17 15:07:14,2022-02-17 15:14:24,1444,1256
694,14545500694,Ghim Moh Market & Food Centre,11,2022-02-17 14:54:55,2022-02-17 15:39:55,2022-02-17 15:06:15,2022-02-17 15:09:12,1843,857
696,14553600696,Ghim Moh Market & Food Centre,11,2022-02-17 14:55:36,2022-02-17 15:40:36,2022-02-17 15:06:48,2022-02-17 15:13:26,1630,1070
697,14560900697,Ghim Moh Market & Food Centre,11,2022-02-17 14:56:09,2022-02-17 15:41:09,2022-02-17 15:04:04,2022-02-17 15:12:28,1721,979


In [408]:
new_test_df['num_orders'] = 1
new_test_df[['pickup_id', 'time_period_id', 'num_orders']].groupby(['pickup_id', 'time_period_id'], as_index=False).sum()

,pickup_id,time_period_id,num_orders
0,Alexandra Village Food Centre,4,19
1,Alexandra Village Food Centre,5,26
2,Alexandra Village Food Centre,6,28
3,Alexandra Village Food Centre,7,24
4,Alexandra Village Food Centre,8,14
5,Alexandra Village Food Centre,9,11
6,Alexandra Village Food Centre,10,9
7,Alexandra Village Food Centre,11,10
8,Bukit Timah Market & Food Centre,4,20
9,Bukit Timah Market & Food Centre,5,27


In [250]:
new_test_df[['pickup_id', 'time_period_id', 'overdue']].groupby(['pickup_id', 'time_period_id'], as_index=False).mean()

,pickup_id,time_period_id,overdue
0,Alexandra Village Food Centre,4,1542.736842
1,Alexandra Village Food Centre,5,805.115385
2,Alexandra Village Food Centre,6,21.035714
3,Alexandra Village Food Centre,7,424.583333
4,Alexandra Village Food Centre,8,499.928571
5,Alexandra Village Food Centre,9,916.818182
6,Alexandra Village Food Centre,10,1447.444444
7,Alexandra Village Food Centre,11,1764.000000
8,Bukit Timah Market & Food Centre,4,1388.600000
9,Bukit Timah Market & Food Centre,5,133.444444
